#  Central Bank Score

## Análise de Sentimentos em Discursos de Bancos Centrais

Este projeto realiza análise de sentimentos em discursos de Bancos Centrais de diversos países,
gerando um **score Hawkish-Dovish** para simplificar a interpretação econômica dos comunicados.

### Objetivos
- Identificar tópicos principais nos discursos usando **LDA**
- Analisar sentimentos com **VADER**
- Calcular índice de tom (Hawkish vs Dovish)
- Visualizar tendências por país e período

###  Dataset
- **Fonte**: Kaggle - Central Bank Speeches
- **Tamanho**: ~7.700 discursos em inglês
- **Período**: Vários anos de discursos de Bancos Centrais globais

In [ ]:
#  Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
import os

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from wordcloud import WordCloud
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Configurar estilo dos gráficos
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print('✅ Bibliotecas importadas com sucesso!')

In [ ]:
#  Download dos recursos NLTK necessários
nltk_resources = [
    'stopwords',
    'punkt',
    'punkt_tab',  
    'wordnet',
    'vader_lexicon',
    'averaged_perceptron_tagger',  
    'omw-eng'  # Open Multilingual WordNet 
]

print('📥 Baixando recursos do NLTK...')
print('=' * 40)

for resource in nltk_resources:
    try:
        nltk.download(resource, quiet=False)  # quiet=False para ver progresso
        print(f'✅ {resource} baixado com sucesso!')
    except Exception as e:
        print(f'⚠️ Erro ao baixar {resource}: {e}')

print('=' * 40)
print('✅ Recursos NLTK baixados!')

In [ ]:
#  Carregar a base de dados
file_path = 'all_speeches.csv'

# Verificar se o arquivo existe
if os.path.exists(file_path):
    df = pd.read_csv(file_path)
    print(f'✅ Dataset carregado com sucesso!')
    print(f'📊 Dimensões: {df.shape[0]} linhas x {df.shape[1]} colunas')
else:
    print(f'❌ Arquivo não encontrado: {file_path}')
    print('💡 Certifique-se de que o arquivo all_speeches.csv está na pasta correta')

# Converter a coluna de data para datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Criar colunas adicionais para análise temporal
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['year_month'] = df['date'].dt.to_period('M').astype(str)

# Visualizar primeiras linhas
df.head()

In [ ]:
# ℹ️ Informações sobre o dataset
print('📋 Informações do Dataset:')
print('=' * 50)
df.info()

print('\n📈 Estatísticas por País:')
print('=' * 50)
print(df['country'].value_counts())

print('\n📅 Período dos Dados:')
print('=' * 50)
print(f'Data inicial: {df["date"].min()}')
print(f'Data final: {df["date"].max()}')

## Pré-processamento dos Textos

Nesta etapa, realizamos:
1. Remoção de pontuações
2. Remoção de stopwords
3. Lemmatização (redução à raiz das palavras)

In [ ]:
# Função para remover pontuação
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', str(text))

# Aplicar limpeza
corpus = df['text']
clean_corpus = corpus.apply(lambda x: remove_punctuation(x))

# Classe para Lemmatização
class Lemmatizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    
    def __call__(self, text):
        return [self.wnl.lemmatize(word) for word in word_tokenize(text)]

# Carregar e processar stop words
lemmatizer = Lemmatizer()
stop_words = set(stopwords.words('english'))
processed_stop_words = set()

for word in stop_words:
    processed_stop_words.update(lemmatizer(word))

print(f'✅ Pré-processamento concluído!')
print(f'📊 Stopwords processadas: {len(processed_stop_words)}')

In [ ]:
# 🔢 Vetorização do texto
MAX_VOCAB_SIZE = 40000

vectorizer = CountVectorizer(
    tokenizer=Lemmatizer(),
    stop_words=list(processed_stop_words),
    max_features=MAX_VOCAB_SIZE
)

data = vectorizer.fit_transform(clean_corpus)

print(f'✅ Vetorização concluída!')
print(f'📊 Shape da matriz: {data.shape}')

## Modelagem de Tópicos com LDA

O **Latent Dirichlet Allocation (LDA)** é um modelo não-supervisionado que identifica
tópicos ocultos em uma coleção de documentos.

Identificando **5 tópicos principais** nos discursos dos Bancos Centrais.

In [ ]:
# Aplicar LDA
lda = LatentDirichletAllocation(
    n_components=5,
    random_state=1234,
    max_iter=10
)

lda.fit(data)

print('✅ Modelo LDA treinado!')
print('\n📋 Palavras mais frequentes por tópico:')
print('=' * 60)

for idx, topic in enumerate(lda.components_):
    top_words = [vectorizer.get_feature_names_out()[i] for i in topic.argsort()[:-20 - 1:-1]]
    print(f'\n📝 Tópico {idx}:')
    print(', '.join(top_words))

In [ ]:
#  Visualização com WordCloud
def plot_wordcloud(model, feature_names, n_words=20):
    for topic_idx, topic in enumerate(model.components_):
        words = [feature_names[i] for i in topic.argsort()[:-n_words - 1:-1]]
        wordcloud = WordCloud(
            width=800,
            height=400,
            background_color='white',
            colormap='viridis'
        ).generate(' '.join(words))
        
        plt.figure(figsize=(10, 5))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis('off')
        plt.title(f'WordCloud - Tópico {topic_idx}', fontsize=16)
        plt.tight_layout()
        plt.show()

# Plotar WordClouds
feature_names = vectorizer.get_feature_names_out()
plot_wordcloud(lda, feature_names)

## Análise de Sentimentos com VADER

O **VADER (Valence Aware Dictionary and sEntiment Reasoner)** é uma ferramenta
de análise de sentimentos baseada em léxico e regras, especialmente projetada
para análise de sentimentos em mídias sociais.

Métricas:
- **Compound**: Score geral de sentimento (-1 a +1)
- **Positivo**: Proporção de conteúdo positivo
- **Negativo**: Proporção de conteúdo negativo
- **Neutro**: Proporção de conteúdo neutro

In [ ]:
# Inicializar analisador VADER
sid = SentimentIntensityAnalyzer()

# Função para calcular sentimento
def sentiment_analysis(text):
    return sid.polarity_scores(text)

# Aplicar análise de sentimentos
print('⏳ Aplicando análise de sentimentos (isso pode levar alguns minutos)...')
df['sentiment_scores'] = clean_corpus.apply(lambda x: sentiment_analysis(x))

# Extrair pontuações individuais
df['compound'] = df['sentiment_scores'].apply(lambda x: x['compound'])
df['positive'] = df['sentiment_scores'].apply(lambda x: x['pos'])
df['negative'] = df['sentiment_scores'].apply(lambda x: x['neg'])
df['neutral'] = df['sentiment_scores'].apply(lambda x: x['neu'])

print('✅ Análise de sentimentos concluída!')

In [ ]:
# 📊 Estatísticas de Sentimentos
print('📈 Médias dos Sentimentos:')
print('=' * 40)
print(f'Compound Score: {df["compound"].mean():.4f}')
print(f'Positivo: {df["positive"].mean():.4f} ({df["positive"].mean()*100:.1f}%)')
print(f'Negativo: {df["negative"].mean():.4f} ({df["negative"].mean()*100:.1f}%)')
print(f'Neutro: {df["neutral"].mean():.4f} ({df["neutral"].mean()*100:.1f}%)')

# Visualizar distribuição
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Compound
sns.histplot(df['compound'], bins=30, kde=True, ax=axes[0, 0], color='purple')
axes[0, 0].set_title('Distribuição - Compound Score')
axes[0, 0].set_xlabel('Score')

# Positivo
sns.histplot(df['positive'], bins=30, kde=True, ax=axes[0, 1], color='green')
axes[0, 1].set_title('Distribuição - Sentimento Positivo')
axes[0, 1].set_xlabel('Proporção')

# Negativo
sns.histplot(df['negative'], bins=30, kde=True, ax=axes[1, 0], color='red')
axes[1, 0].set_title('Distribuição - Sentimento Negativo')
axes[1, 0].set_xlabel('Proporção')

# Neutro
sns.histplot(df['neutral'], bins=30, kde=True, ax=axes[1, 1], color='gray')
axes[1, 1].set_title('Distribuição - Sentimento Neutro')
axes[1, 1].set_xlabel('Proporção')

plt.tight_layout()
plt.show()

In [ ]:
# 🎯 Identificar tópico dominante para cada documento
topic_assignments = lda.transform(data)
df['dominant_topic'] = topic_assignments.argmax(axis=1)

# Calcular médias de sentimentos por tópico
topic_sentiment = df.groupby('dominant_topic').agg({
    'compound': 'mean',
    'positive': 'mean',
    'negative': 'mean',
    'neutral': 'mean'
}).reset_index()

# Visualizar
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

metrics = [('compound', 'Compound Score'), ('positive', 'Positivo'),
           ('negative', 'Negativo'), ('neutral', 'Neutro')]

for idx, (col, title) in enumerate(metrics):
    row, col_idx = idx // 2, idx % 2
    sns.barplot(x='dominant_topic', y=col, data=topic_sentiment,
                palette='viridis', ax=axes[row, col_idx])
    axes[row, col_idx].set_title(f'Média - {title} por Tópico')
    axes[row, col_idx].set_xlabel('Tópico Dominante')

plt.tight_layout()
plt.show()

print('\n📊 Tabela - Médias por Tópico:')
print(topic_sentiment.round(4))

## 🦅🕊️ Score Hawkish-Dovish

O score Hawkish-Dovish mede o tom dos discursos dos Bancos Centrais:

- **🦅 Hawkish (+1)**: Banco Central comprometido com metas, decisões previsíveis
  - Palavras: high, strong, increase, fast, accelerate, boom, expansion...

- **🕊️ Dovish (-1)**: Banco Central discricionário, decisões imprevisíveis
  - Palavras: low, weak, decrease, slow, recession, decline, contraction...

**Fórmula:**
\[\text{tone}_{i,t} = \frac{\text{hawkish}_{i,t} - \text{dovish}_{i,t}}{\text{hawkish}_{i,t} + \text{dovish}_{i,t}}\]

In [ ]:
# Definir listas de palavras-chave
hawkish_keywords = [
    'high', 'strong', 'increase', 'fast', 'accelerate', 'better', 'boom', 'emerg', 'expansion',
    'favo(u)rabl', 'firm', 'great', 'improv', 'increas', 'larger', 'positive', 'rais', 'risk',
    'stabiliz', 'stable', 'strengthen', 'strong', 'subdued', 'unsustainable', 'upside',
    'upswing', 'upturn', 'upward'
]

dovish_keywords = [
    'low', 'weak', 'decreas', 'slow', 'collapse', 'contraction', 'dampen', 'decelerate',
    'decline', 'decreas', 'delay', 'depression', 'destabiliz', 'deteriorat', 'difficult',
    'diminish', 'disappear', 'downside', 'downturn', 'downward', 'fall', 'fragile', 'low',
    'negative', 'poor', 'recession', 'slow', 'sluggish', 'small', 'struggling', 'unstable',
    'unfavo(u)rabl', 'wors'
]

# Função para contar palavras-chave
def count_keywords(text, keywords):
    count = 0
    for keyword in keywords:
        count += len(re.findall(keyword, text, re.IGNORECASE))
    return count

# Aplicar contagem
df['hawkish_count'] = clean_corpus.apply(lambda x: count_keywords(x, hawkish_keywords))
df['dovish_count'] = clean_corpus.apply(lambda x: count_keywords(x, dovish_keywords))

# Calcular índice de tom
def calculate_tone(hawkish_count, dovish_count):
    if hawkish_count + dovish_count == 0:
        return np.nan
    return (hawkish_count - dovish_count) / (hawkish_count + dovish_count)

df['tone'] = df.apply(
    lambda row: calculate_tone(row['hawkish_count'], row['dovish_count']),
    axis=1
)

print('✅ Score Hawkish-Dovish calculado!')
print(f'📊 Média geral do índice de tom: {df["tone"].mean():.4f}')

In [ ]:
# 🌍 Análise do índice de tom por país
tone_by_country = df.groupby('country')['tone'].agg(['mean', 'count']).reset_index()
tone_by_country.columns = ['country', 'mean_tone', 'count']
tone_by_country = tone_by_country.sort_values('mean_tone', ascending=False)

print('📊 Média do Índice de Tom por País:')
print('=' * 50)
for _, row in tone_by_country.iterrows():
    print(f"{row['country']:<20} {row['mean_tone']:>8.4f}  ({row['count']:>4} discursos)")

# Visualização
plt.figure(figsize=(12, 6))
sns.barplot(x='country', y='mean_tone', data=tone_by_country, palette='RdYlGn')
plt.title('🦅🕊️ Média do Índice de Tom por País', fontsize=14)
plt.xlabel('País')
plt.ylabel('Índice de Tom Médio')
plt.xticks(rotation=45, ha='right')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

In [ ]:
# 📊 Distribuição do índice de tom por país
g = sns.FacetGrid(df, col='country', col_wrap=4, height=3, aspect=1.2)
g.map(sns.histplot, 'tone', bins=30, kde=True, color='steelblue')
g.set_titles('{col_name}')
g.set_axis_labels('Índice de Tom', 'Frequência')
g.fig.suptitle('Distribuição do Índice de Tom por País', y=1.02, fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# 📈 Série temporal do índice de tom (a partir de 2020)
df_recent = df[df['year'] >= 2020].copy()

temporal_tone = df_recent.groupby('year_month')['tone'].mean().reset_index()

plt.figure(figsize=(14, 6))
sns.lineplot(x='year_month', y='tone', data=temporal_tone, marker='o', linewidth=2)
plt.title('📈 Variação do Índice de Tom ao Longo do Tempo (2020+)', fontsize=14)
plt.xlabel('Período (Ano-Mês)')
plt.ylabel('Índice de Tom Médio')
plt.xticks(rotation=45, ha='right')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5, label='Neutro')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 📈 Evolução do índice de tom por país (1990+)
df_filtered = df[df['year'] >= 1990].copy()

tone_by_year_country = df_filtered.groupby(['year', 'country'])['tone'].mean().unstack()

plt.figure(figsize=(14, 8))
tone_by_year_country.plot(marker='o', linewidth=2, alpha=0.8)
plt.title('📈 Evolução do Índice de Tom por País (1990-2023)', fontsize=14)
plt.xlabel('Ano')
plt.ylabel('Tone Médio')
plt.xticks(rotation=45)
plt.legend(title='País', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 📦 Boxplot do índice de tom por país
plt.figure(figsize=(14, 6))
sns.boxplot(x='country', y='tone', data=df, palette='Set2')
plt.title('📦 Distribuição do Índice de Tom por País', fontsize=14)
plt.xlabel('País')
plt.ylabel('Índice de Tom')
plt.xticks(rotation=45, ha='right')
plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()

## Conclusões

### Principais Achados:

1. **Sentimentos**: Os discursos dos Bancos Centrais apresentam predominantemente
   caráter **neutro** (~80%), com inclinação positiva (~13%) maior que negativa (~6%)

2. **Tópicos**: Os tópicos mais relevantes para interpretação econômica incluem:
   - Política monetária e taxas de juros
   - Inflação e estabilidade de preços
   - Crescimento econômico

3. **Índice de Tom**: A média geral está próxima de zero, indicando neutralidade,
   com leve tendência hawkish na maioria dos países

4. **Variação por País**: Diferentes Bancos Centrais apresentam padrões distintos
   de comunicação, refletindo suas estratégias e contextos econômicos

### Possíveis Melhorias Futuras:
- Aplicar modelos mais avançados (BERT, LSTM)
- Expandir análise para mais idiomas
- Incorporar dados de mercado para validação